Text Annotation with spaCy
======================

TODO: introduce language models and spaCy's take on them

spaCy Language Models
-----------------------------

In [1]:
import spacy

nlp = spacy.load('en_core_web_md')

Annotations
--------------

To annotate a document with the `spaCy` model, simply run it through the core function, `nlp()`. We'll do so with a short poem by Gertrude Stein.

In [2]:
with open('data/session_one/stein_carafe.txt', 'r') as f:
    stein_poem = f.read()
    
carafe = nlp(stein_poem)

With this done, we can inspect the result...

In [3]:
carafe

A kind in glass and a cousin, a spectacle and nothing strange a single hurt color and an arrangement in a system to pointing. All this and not ordinary, not unordered in not resembling. The difference is spreading.

...which seems to be no different from a string representation! This output is a bit misleading, however. Our `carafe` object actually has a ton of extra information associated with it, even though, on the surface, it appears to be a plain old string.

If you'd like, you can inspect all these attributes and methods with:

In [4]:
attributes = [i for i in dir(carafe) if i.startswith("_") is False]

We won't show them all here, but suffice it to say, there are a lot!

In [5]:
print("Number of attributes in a SpaCy doc:", len(attributes))

Number of attributes in a SpaCy doc: 51


This high number of attributes indicates an important point to keep in mind when working with `spaCy`: as opposed to other forms of text preprocessing, like changing the case of tokens, removing punctuation, or stemming a corpus, `spaCy` aims to **maximally preserve information** about your document. It keeps documents intact and in fact adds much more information about them than Python's base string methods have. In this sense, we might say that `spaCy` is additive in nature, whereas text mining methods are subtractive, or reductive.

### Document Annotations

So, while the base representation of `carafe` looks like a string, under the surface there are all sorts of annotations about it. To access them, we use the attributes counted above. For example, `spaCy` adds extra segmentation information about a document, like which parts of it belong to different sentences. We can check to see whether this information has been attached to our text with the `.has_annotation()` method.

In [6]:
carafe.has_annotation('SENT_START')

True

We can use the same method to check for a few other annotations:

In [7]:
annotation_types = {'Dependencies': 'DEP', 'Entities': 'ENT_IOB', 'Tags': 'TAG'}
for a, t in annotation_types.items():
    print(
        f"{a:>12}: {carafe.has_annotation(t)}"
    )

Dependencies: True
    Entities: True
        Tags: True


Let's look at sentences. We can access them with `.sents`.

In [8]:
carafe.sents

...but you can see that there's a small complication here: `.sents` returns a generator, not a list. The reason has to do with memory efficiency. Because `SpaCy` adds so much extra information about your document, this information could slow down your code or overwhelm your computer if the library didn't store it in an efficient manner. Of course this isn't a problem with our small poem, but you can imagine how it could become one with a big corpus.

To access the actual sentences in `carafe`, we'll need to convert the generator to a list.

```{margin} Want to learn more about generators?
The DataLab has a workshop about them. See [this link].

[this link]: https://datalab.ucdavis.edu/eventscalendar/intermediate-python-iterator-generator-crash-course/
```

In [9]:
import textwrap

sentences = list(carafe.sents)
for s in sentences:
    s = textwrap.shorten(s.text, width=100)
    print(s)

A kind in glass and a cousin, a spectacle and nothing strange a single hurt color and an [...]
All this and not ordinary, not unordered in not resembling.
The difference is spreading.


One very useful attribute is `.noun_chunks`. It returns nouns and compound nouns in a document.

In [10]:
noun_chunks = list(carafe.noun_chunks)

for noun in noun_chunks:
    print(noun)

A kind
glass
a cousin
a spectacle
nothing
a single hurt color
an arrangement
a system
The difference


See how this picks up not only nouns, but articles and compound information? Articles could be helpful if you wanted to track singular/plural relationships, while compound nouns might tell you something about the way a document refers to the entities therein. The latter could have repeating patterns, and you might imagine how you could use noun chunks to create and count n-gram tokens and feed that into a classifier.

Consider this example from _The Odyssey_. Homer used many epithets and repeating phrases throughout his epic. According to some theories, these act as mnemonic devices, helping a performer keep everything in their head during an oral performance (the poem wasn't written down in Homer's day). Using `.noun_chunks` in conjunction with a Python `Counter`, we may be able to identify these in Homer's text. Below, we'll do so with _The Odyssey_ Book XI.

First, let's load and model the text.

In [11]:
with open('data/session_one/odyssey_book_11.txt', 'r') as f:
    book_eleven = f.read()
    
odyssey = nlp(book_eleven)

Now we'll import a `Counter` and initialize it. Then we'll get the noun chunks from the document and populate them in the count dictionary with a list comprehension line. Be sure to only grab the text from each token. We'll explain why in a little while.

In [12]:
from collections import Counter

noun_counts = Counter([chunk.text for chunk in odyssey.noun_chunks])

With that done, let's look for repeating noun chunks with three or more words.

```{margin} What we're doing here...
For every noun chunk in the counter:

1. Split the chunk
2. Check if the length of the chunk is more than two and the count is more than one
3. If so, join the chunk back together and print it along with the chunk
```

In [13]:
print("Phrase                  | Count")
print("-" * 31)
for chunk, count in noun_counts.items():
    chunk = chunk.split()
    if (len(chunk) > 2) and (count > 1):
        joined = ' '.join(chunk)
        print(f"{joined:<25}{count:>4}")

Phrase                  | Count
-------------------------------
the sea shore               2
a fair wind                 2
the poor feckless ghosts    2
the same time               2
the other side              2
his golden sceptre          2
your own house              2
her own son                 2
the Achaean land            2
her own husband             2
my wicked wife              2
all the Danaans             2
the poor creature           2


Excellent! Looks like we turned up a few: "the poor feckless ghosts," "my wicked wife," and "all the Danaans" are likely the kind of repeating phrases scholars think of in Homer's text.

Another way to look at entities in a text is with `.ents`. `spaCy` uses **named-entity recognition** to extract significant objects, or entities, in a document. In general, anything that has a proper name associated with it is considered an entity, but things expressions of time and geographic location are also often tagged. Here are the first five from Book XI above.

In [14]:
entities = list(odyssey.ents)

count = 0
while count < 5:
    print(entities[count])
    count += 1

Circe
Oceanus
Cimmerians
Circe
Perimedes


You can select particular entities using the `.label_` attribute. Here are all the temporal entities in Book XI.

In [15]:
[e.text for e in odyssey.ents if e.label_ == 'TIME']

['all night', 'to-morrow morning', 'the light of day']

And here is a unique listing of all the people.

```{margin} How many labels are there?
This will depend on the model. Here's the [label scheme] for the one we're using.

[label scheme]: https://spacy.io/models/en#en_core_web_md-labels
```

In [16]:
set(e.text for e in odyssey.ents if e.label_ == 'PERSON')

{'Achilles',
 'Aeson',
 'Alcinous',
 'Arete',
 'Ariadne',
 'Cassandra',
 'Chloris',
 'Circe',
 'Clytemnestra',
 'Diana',
 'Echeneus',
 'Epicaste',
 'Eriphyle',
 'Eurylochus',
 'Helen',
 'Iphicles',
 'Iphimedeia',
 'Jove',
 'Leda',
 'Leto',
 'Maera',
 'Megara',
 'Memnon',
 'Minerva',
 'Neleus',
 'Neoptolemus',
 'Nestor',
 'OEdipodes',
 'Orestes',
 'Ossa',
 'Periclymenus',
 'Perimedes',
 'Pero',
 'Pollux',
 'Priam',
 'Proserpine',
 'Pylos',
 'Pytho',
 'Queen',
 'Scyros',
 'Sisyphus',
 'Teiresias',
 'Telemachus',
 'Theban Teiresias',
 'Ulysses'}

### Token Annotations

In addition to storing all of this information about texts, `spaCy` creates a substantial amount of annotations for each of the tokens in that document. The same logic as above applies to accessing this information.

Let's return to the Stein poem. Indexing `carafe` will return individual tokens:

In [17]:
carafe[3]

glass

Like `carafe`, each one has several attributes:

In [18]:
token_attributes = [i for i in dir(carafe[3]) if i.startswith("_") is False]

print("Number of token attributes:", len(token_attributes))

Number of token attributes: 94


That's a lot!

These attributes range from simple booleans, like whether a token is an alphabetic character:

In [19]:
carafe[3].is_alpha

True

...or whether it is a stop word:

In [20]:
carafe[3].is_stop

False

...to more complex pieces of information, like tracking back to the sentence this token is part of:

In [21]:
carafe[3].sent

A kind in glass and a cousin, a spectacle and nothing strange a single hurt color and an arrangement in a system to pointing.

...sentiment scores:

In [22]:
carafe[3].sentiment

0.0

...and even vector space representations:

In [23]:
carafe[3].vector

array([-1.4859e-01, -1.7940e-01,  4.3666e-02,  1.5748e-01,  1.3568e-01,
       -9.3666e-01, -6.8430e-01,  4.7692e-01, -4.1391e-01,  9.3575e-01,
       -1.6360e-01,  6.7553e-02, -2.7843e-01, -5.6125e-01,  1.3088e-01,
       -1.0006e-01,  7.0374e-03,  2.6217e+00,  5.4600e-02, -5.8931e-01,
        2.5739e-04, -2.6791e-01,  4.6093e-01, -5.9145e-02, -1.0330e-01,
       -3.7589e-01, -2.5343e-01,  1.4790e-02, -4.8031e-01, -4.4314e-01,
        2.4685e-01, -8.6519e-04, -1.2361e-01,  9.1683e-02, -1.5880e-01,
       -4.5974e-01,  3.3017e-01, -4.4124e-01,  3.3604e-01, -3.0438e-01,
        4.4664e-01,  2.2697e-01,  2.9327e-02, -2.7025e-01,  3.1813e-01,
       -1.5890e-01, -4.1371e-01, -9.0721e-01, -2.0866e-01,  3.6400e-01,
        5.6862e-02, -2.6824e-01, -2.9722e-01,  6.2107e-02, -4.7908e-01,
       -5.8164e-01, -1.4302e-01,  7.0109e-02, -1.2735e-01,  3.6194e-02,
       -1.6634e-01, -2.2135e-01, -5.0446e-02,  4.3839e-01, -5.5363e-01,
       -4.4219e-01, -1.3657e-01, -2.8472e-01, -5.0637e-01,  7.99

Here's a listing of some attributes you might want to know about when text mining.

In [24]:
import pandas as pd

sample_attributes = []

for token in carafe:
    sample_attributes.append({
        'TEXT': token.text,
        'INDEX': token.i,
        'LOWERCASE': token.lower_,
        'ALPHABETIC': token.is_alpha,
        'DIGIT': token.is_digit,
        'PUNCTUATION': token.is_punct,
        'STARTS SENTENCE': token.is_sent_start,
        'LIKE URL': token.like_url
    })

sample_attributes = pd.DataFrame(sample_attributes).head(10)
sample_attributes.style.hide_index()

TEXT,INDEX,LOWERCASE,ALPHABETIC,DIGIT,PUNCTUATION,STARTS SENTENCE,LIKE URL
A,0,a,True,False,False,True,False
kind,1,kind,True,False,False,False,False
in,2,in,True,False,False,False,False
glass,3,glass,True,False,False,False,False
and,4,and,True,False,False,False,False
a,5,a,True,False,False,False,False
cousin,6,cousin,True,False,False,False,False
",",7,",",False,False,True,False,False
a,8,a,True,False,False,False,False
spectacle,9,spectacle,True,False,False,False,False


We'll discuss some of the more complex annotations later on, both in this session and others. For now, let's collect some simple information about each of the tokens in our document. We'll use list comprehension to do so. We'll also use the `.text` attribute for each token, since we only want the text representation. Otherwise, we'd be creating a list of generators, where each generator has all those attribute for every token! (This is why we made sure to only use `.text` in our work with _The Odyssey_ above.)

In [25]:
words = ' '.join([token.text for token in carafe if token.is_alpha])
punctuation = ' '.join([token.text for token in carafe if token.is_punct])

print(
    f"Words\n-----\n{textwrap.shorten(words, width=100)}",
    f"\n\nPunctuation\n-----------\n{punctuation}"
)

Words
-----
A kind in glass and a cousin a spectacle and nothing strange a single hurt color and an [...] 

Punctuation
-----------
, . , . .


Want some linguistic information? We can get that too. For example, here are prefixes and suffixes:

```{margin} You might be wondering about those underscores...

The syntax conventions of `spaCy` use an underscore to access the actual attribute information for a token. Using an attribute without the underscore will return an id, which the library uses internally to piece together output.
```

In [26]:
prefix_suffix = [(token.text, token.prefix_, token.suffix_) for token in carafe if token.is_alpha]

for i in prefix_suffix:
    text, prefix, suffix = i[0], i[1], i[2]
    print(
        f"Text: {text}\n+ Prefix: {prefix}\n+ Suffix: {suffix}\n"
    )

Text: A
+ Prefix: A
+ Suffix: A

Text: kind
+ Prefix: k
+ Suffix: ind

Text: in
+ Prefix: i
+ Suffix: in

Text: glass
+ Prefix: g
+ Suffix: ass

Text: and
+ Prefix: a
+ Suffix: and

Text: a
+ Prefix: a
+ Suffix: a

Text: cousin
+ Prefix: c
+ Suffix: sin

Text: a
+ Prefix: a
+ Suffix: a

Text: spectacle
+ Prefix: s
+ Suffix: cle

Text: and
+ Prefix: a
+ Suffix: and

Text: nothing
+ Prefix: n
+ Suffix: ing

Text: strange
+ Prefix: s
+ Suffix: nge

Text: a
+ Prefix: a
+ Suffix: a

Text: single
+ Prefix: s
+ Suffix: gle

Text: hurt
+ Prefix: h
+ Suffix: urt

Text: color
+ Prefix: c
+ Suffix: lor

Text: and
+ Prefix: a
+ Suffix: and

Text: an
+ Prefix: a
+ Suffix: an

Text: arrangement
+ Prefix: a
+ Suffix: ent

Text: in
+ Prefix: i
+ Suffix: in

Text: a
+ Prefix: a
+ Suffix: a

Text: system
+ Prefix: s
+ Suffix: tem

Text: to
+ Prefix: t
+ Suffix: to

Text: pointing
+ Prefix: p
+ Suffix: ing

Text: All
+ Prefix: A
+ Suffix: All

Text: this
+ Prefix: t
+ Suffix: his

Text: and
+ Prefix: a
+

And here are lemmas:

In [27]:
lemmas = {token.text: token.lemma_ for token in carafe if token.is_alpha}

print("Text         | Lemma")
print("-" * 28)
for text, lemma in lemmas.items():
    print(
        f"{text:<15}{lemma}"
    )

Text         | Lemma
----------------------------
A              a
kind           kind
in             in
glass          glass
and            and
a              a
cousin         cousin
spectacle      spectacle
nothing        nothing
strange        strange
single         single
hurt           hurt
color          color
an             an
arrangement    arrangement
system         system
to             to
pointing       point
All            all
this           this
not            not
ordinary       ordinary
unordered      unordere
resembling     resemble
The            the
difference     difference
is             be
spreading      spread


With such attributes at your disposal, you might imagine how you could work `spaCy` into a text mining pipeline. Instead of using separate functions to clean your corpus, those steps could all be accomplished by accessing attributes.

Before you do this, however, you should consider two things: 1) whether the increased computational/memory overhead is worthwhile for your project; and 2) whether `spaCy`'s base models will work for the kind of text you're using. This second point is especially important. While `spaCy`'s base models are incredibly powerful, they are built for general purpose applications and may struggle with domain-specific language. Medical text and early modern print are two such examples of where the base models interpret your documents in unexpected ways, thereby complicating, maybe even ruining, parts of a text mining pipeline that relies on them. Sometimes, in other words, it's just best to stick with a text mining pipeline that you know to be effective.

That all said, there are ways to train your own `spaCy` model on a specific domain. This can be an extensive process, one which exceeds the limits of our short workshop, but if you want to learn more about doing so, you can visit [this page]. There are also [third party models] available, which you might find useful, though your milage may vary.

[this page]: https://spacy.io/usage/training
[third party models]: https://spacy.io/universe/category/models

Part-of-Speech Tagging
----------------------------

One of the most common tasks in NLP involves assigning **part-of-speech, or POS, tags** to each token in a document. As we saw in the text mining series, these tags are a necessary step for certain text cleaning process, like lemmatization; you might also use them to identify subsets of your data, which you could separate out and model. Beyond text cleaning, POS tags can be useful for tasks like **word sense disambiguation**, where you try to determine which particular facet of meaning a given token represents.

Regardless of the task, the process of getting POS tags from `spaCy` will be the same. Each token in a document has an associated tag, which is accessible as an attribute.

In [28]:
pos = {token.text: token.pos_ for token in carafe}

print("Text        | POS Tag")
print("-" * 21)
for text, tag in pos.items():
    print(f"{text:<15}{tag}")

Text        | POS Tag
---------------------
A              DET
kind           NOUN
in             ADP
glass          NOUN
and            CCONJ
a              DET
cousin         NOUN
,              PUNCT
spectacle      NOUN
nothing        PRON
strange        ADJ
single         ADJ
hurt           NOUN
color          NOUN
an             DET
arrangement    NOUN
system         NOUN
to             ADP
pointing       VERB
.              PUNCT
All            DET
this           DET
not            PART
ordinary       ADJ
unordered      VERB
resembling     VERB
The            DET
difference     NOUN
is             AUX
spreading      VERB


If you don't know what a tag means, you can use `spacy.explain()`.

In [29]:
spacy.explain('CCONJ')

'coordinating conjunction'

`spaCy` actually has two types of POS tags. The ones accessible with the `.pos_` attribute are the basic tags, whereas those under `.tag_` are more detailed (these come from the [Penn Treebank project]). We'll print them out below, along with information about what they mean.

[Penn Treebank project]: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [30]:
detailed_tags = {token.text: token.tag_ for token in carafe}

print("Text        | POS Tag | Explanation")
print("-" * 74)
for text, tag in detailed_tags.items():
    explanation = spacy.explain(tag)
    print(
        f"{text:<16}{tag:<8}{explanation}"
    )

Text        | POS Tag | Explanation
--------------------------------------------------------------------------
A               DT      determiner
kind            NN      noun, singular or mass
in              IN      conjunction, subordinating or preposition
glass           NN      noun, singular or mass
and             CC      conjunction, coordinating
a               DT      determiner
cousin          NN      noun, singular or mass
,               ,       punctuation mark, comma
spectacle       NN      noun, singular or mass
nothing         NN      noun, singular or mass
strange         JJ      adjective (English), other noun-modifier (Chinese)
single          JJ      adjective (English), other noun-modifier (Chinese)
hurt            NN      noun, singular or mass
color           NN      noun, singular or mass
an              DT      determiner
arrangement     NN      noun, singular or mass
system          NN      noun, singular or mass
to              IN      conjunction, subordinat

This is all well and good in the abstract, but the power of POS tags lies in how they support other kinds of analysis. We'll do a quick word sense disambiguation task here but will return to do something more complex in a little while.

Between the two strings:

1. "I am not going to bank on that happening."
2. "I went down to the river bank."

How can we tell which sense of the word "bank" is being used? Well, we can model each with `spaCy` and see whether the POS tags for these two tokens match. If they don't match, this will indicate that the tokens represent two different senses of the word "bank."

All this can be accomplished with a `for` loop and `nlp.pipe()`. The latter function enables you to process different documents with the `spaCy` model all at once. This can be great for working with a large corpus, though note that, because `.pipe()` is meant to work on text at scale, it will return a generator.

In [31]:
banks = ["I am not going to bank on that happening.", "I went down to the river bank."]

nlp.pipe(banks)

<generator object Language.pipe at 0x12eef10c0>

In [32]:
for doc in nlp.pipe(banks):
    for token in doc:
        if token.text == 'bank':
            print(
                f"{doc.text}\n+ {token.text}: "
                f"{token.tag_} ({spacy.explain(token.tag_)})\n"
            )

I am not going to bank on that happening.
+ bank: VB (verb, base form)

I went down to the river bank.
+ bank: NN (noun, singular or mass)



Dependency Parsing
------------------------

TODO: a discussion of dependency parsing.

In [33]:
from spacy import displacy

to_render = list(carafe.sents)[2]
displacy.render(to_render, style='dep')

Seeing these relationships are quite useful in and of themselves, but the real power of dependency parsing comes in all the extra data it can provide about a token. Using this technique, you can link tokens back to their heads, or find local groupings of tokens that all refer to the same head.

Here's how you could formalize that with a dataframe. Given this sentence:

In [34]:
sentence = odyssey[2246:2260]
sentence.text

"Then I tried to find some way of embracing my mother's ghost."

We can construct a `for` loop, which rolls through each token and retrieves its dependency info.

In [35]:
dependencies = []
for token in sentence:
    dependencies.append({
        'INDEX': token.i,
        'TOKEN': token.text,
        'DEPENDENCY_SHORTCODE': token.dep_,
        'DEPENDENCY': spacy.explain(token.dep_),
        'HEAD_INDEX': token.head.i,
        'HEAD': token.head
    })
    
dependencies = pd.DataFrame(dependencies)
dependencies.style.hide_index()

INDEX,TOKEN,DEPENDENCY_SHORTCODE,DEPENDENCY,HEAD_INDEX,HEAD
2246,Then,advmod,adverbial modifier,2248,tried
2247,I,nsubj,nominal subject,2248,tried
2248,tried,ROOT,None,2248,tried
2249,to,aux,auxiliary,2250,find
2250,find,xcomp,open clausal complement,2248,tried
2251,some,det,determiner,2252,way
2252,way,dobj,direct object,2250,find
2253,of,prep,prepositional modifier,2252,way
2254,embracing,pcomp,complement of preposition,2253,of
2255,my,poss,possession modifier,2256,mother


How many tokens are associated with each head?

In [36]:
dependencies.groupby('HEAD').size()

HEAD
tried        5
find         2
way          2
of           1
embracing    1
mother       2
ghost        1
dtype: int64

Which tokens are in each of these groups?

In [37]:
print("Head     | Group")
print("-" * 46)
for group in dependencies.groupby('HEAD'):
    head, tokens = group[0].text, group[1]['TOKEN'].tolist()
    print(f"{head:<10}{tokens}")

Head     | Group
----------------------------------------------
tried     ['Then', 'I', 'tried', 'find', '.']
find      ['to', 'way']
way       ['some', 'of']
of        ['embracing']
embracing ['ghost']
mother    ['my', "'s"]
ghost     ['mother']


`sciPy` also has a special `.subtree` attribute for each token, which will also produce a similar set of local groupings. Note however that `.subtree` captures all tokens that hold a dependent relationship with the one in question, meaning that when you find the subtree of the root, you're going to print out the entire sentence.

As you might expect by now, `.subtree` returns a generator, so convert it to a list or use list comprehension to extract the tokens. We'll do this in a separate function. Within this function, we're going to use the `.text_with_ws` attribute of each token in the subtree to return an exact, string-like representation of the tree (this will include any whitespace characters that are attached to a token).

In [38]:
def subtree_to_text(subtree):
    subtree = [token.text_with_ws for token in token.subtree]
    return ''.join(subtree)

print("Head      | Dep.  | Subtree")
print("-" * 82)
for token in sentence:
    subtree = subtree_to_text(token.subtree)
    subtree = subtree.strip()
    print(f"{token.text:<12}{token.dep_:<8}{subtree}")

Head      | Dep.  | Subtree
----------------------------------------------------------------------------------
Then        advmod  Then
I           nsubj   I
tried       ROOT    "Then I tried to find some way of embracing my mother's ghost.
to          aux     to
find        xcomp   to find some way of embracing my mother's ghost
some        det     some
way         dobj    some way of embracing my mother's ghost
of          prep    of embracing my mother's ghost
embracing   pcomp   embracing my mother's ghost
my          poss    my
mother      poss    my mother's
's          case    's
ghost       dobj    my mother's ghost
.           punct   .


Putting Everything Together
---------------------------------

Now that we've walked through all these options (which are really only a small sliver of what you can do with `spaCy`!), let's put them into action. Below, we'll construct two short examples of how you might combine different aspects of token attributes to analyze a text.

### Finding lemmas

In the first, we'll use the `.lemma_` attribute to search through Book XI of _The Odyssey_ and match its tokens to a few key words. If you've read _The Odyssey_, you'll know that Book XI is where Odysseus and his fellow sailors have to travel down to the underworld Hades, where they speak with the dead. We already saw one example of this: Odysseus attempts to embrace his dead mother after communing with her. The whole trip to Hades is an emotionally tumultuous experience for the travelers, and peppered throughout Book XI are expressions of grief.

With `.lemma`, we can search for these expressions. We'll roll through the text and determine whether a token lemma matches one of a selected set. When we find a match, we'll get the subtree of this token's _head_. That is, we'll find the head upon which this token depends, and then we'll use that to reconstruct the local context for the token.

In [39]:
for token in odyssey:
    if token.lemma_ in ('cry', 'grief', 'grieve', 'sad', 'sorrow', 'tear', 'weep'):
        subtree = subtree_to_text(token.head.subtree)
        print(token.text)
        print(f"+ {subtree}\n")

weeping
+ weeping and in great distress of mind

cried
+ cried when I saw him: 'Elpenor

sad
+ sad 

tears
+ tears 

sorrow
+ all my sorrow 

sad
+ sad 

tears
+ tears 

grieves
+ He grieves continually about your never having come home, and suffers more and more as he grows older. 

sad
+ sad 

sorrows
+ our sorrows 

grief
+ grief 

grief
+ great grief 

grief
+ grief

sadder
+ still sadder 

weeping
+ weeping 

wept
+ I too wept and pitied him as I beheld him. '

weeping
+ weeping and talking thus sadly with one another 

tear
+ a tear 

cries
+ such appalling cries



### Verb-subject relations

For this next example, we'll use dependency tags to find the subject sentences in Book XI. As before, we'll go through each token in the document, this time checking to see whether it has the `nsubj` or `nsubjpass` tag for its `.dep_` attribute. These refer to the subjects of the sentence's root. We'll also check to see whether a token is a noun (otherwise we'd get a lot of articles like 'who', 'them', etc.). If a token matches these two conditions, we'll find its head verb as well as the token's subtree. Note that this time, the subtree will refer directly to the token in question, not to the head. This will let us capture some descriptive information about each sentence subject.

In [40]:
nsubj = []
for token in odyssey:
    if token.dep_ in ('nsubj', 'nsubjpass') and token.pos_ in ('NOUN', 'PROPN'):
        nsubj.append({
            'SUBJECT': token.text,
            'HEAD': token.head.text,
            'HEAD_LEMMA': token.head.lemma_,
            'SUBTREE': subtree_to_text(token.subtree)
        })

nsubj_df = pd.DataFrame(nsubj)
nsubj_df.head(25).style.hide_index()

SUBJECT,HEAD,HEAD_LEMMA,SUBTREE
Circe,sent,send,"Circe, that great and cunning goddess"
sails,were,be,her sails
sun,went,go,the sun
darkness,was,be,darkness
rays,pierce,pierce,the rays of the sun
wretches,live,live,the poor wretches
Circe,told,tell,Circe
Perimedes,held,hold,Perimedes and Eurylochus
Teiresias,have,have,Teiresias
blood,run,run,the blood


How many time do each of our selected subjects appear?

In [41]:
nsubj_df.groupby('SUBJECT').size().sort_values(ascending=False).head(25)

SUBJECT
ghost           8
Ulysses         5
ghosts          4
heaven          4
wife            4
Proserpine      4
man             4
one             4
Alcinous        3
people          3
Neleus          2
ship            2
judgement       2
Theseus         2
Teiresias       2
gods            2
mother          2
life            2
wind            2
Circe           2
Clytemnestra    2
Hercules        2
Jove            2
creature        2
prisoners       1
dtype: int64

What heads are associated with each subject? (Note that we're using the lemmatized form of the verbs.)

In [42]:
nsubj_df.groupby(['SUBJECT', 'HEAD_LEMMA']).size().sort_values(ascending=False).head(25)

SUBJECT     HEAD_LEMMA
ghost       come          3
Ulysses     answer        2
Proserpine  send          2
one         invite        1
man         do            1
            kill          1
mother      answer        1
            come          1
one         be            1
            get           1
Aegisthus   be            1
man         be            1
one         tell          1
others      fall          1
people      be            1
            bless         1
            hear          1
man         cross         1
limbs       fail          1
property    be            1
heaven      take          1
ground      reek          1
guest       be            1
guests      sit           1
hardship    reach         1
dtype: int64